# 01_DATA_QC_PREP — Quality Check y Splits Estratificados

**Qué hace este notebook:**
1. Carga y normaliza cada `driving_log.csv` de los recorridos seleccionados (`RUNS`).
2. Normaliza rutas de imágenes (soporta barras invertidas y recorta a `IMG/...`).
3. Filtra filas sin imágenes válidas (`center/left/right`).
4. Hace **split estratificado por bins de `steering`** (reproducible).
5. Escribe:
   - `data/processed/<run>/canonical.csv`
   - `data/processed/<run>/{train,val,test}.csv`
   - `data/processed/tasks.json`

---

## 🎯 Objetivos
- Preparar los *splits* `train/val/test` por recorrido
- Generar `tasks.json` para el pipeline *continual*.

## ✅ Prerrequisitos
- `data/raw/udacity/<run>/driving_log.csv`
- Directorio `data/raw/udacity/<run>/IMG/` con las imágenes.

## ⚠️ Salidas
- CSVs canónicos y de split en `data/processed/<run>/`.
- `tasks.json` apuntando a esos CSVs.

> Si más adelante quieres **balanceo offline** de `train` por bins, usa el notebook `01A_PREP_BALANCED.ipynb`.

<a id="toc"></a>

## 🧭 Índice
1. [Entorno y API de preparación](#sec-01)
2. [Parámetros de la preparación](#sec-02)
3. [Ejecutar pipeline y verificar](#sec-03)
4. [Resumen de rutas de splits (vista rápida)](#sec-04)


## <a id="sec-01"></a>

## 1) Entorno y API de preparación

**Objetivo:** preparar el entorno y cargar la API de *prep*.

- Detecta la raíz del repo (`ROOT`) y la añade a `sys.path`.
- Importa funciones del módulo `src.prep.data_prep`:
  - `PrepConfig` — configuración declarativa del *prep*.
  - `run_prep` — ejecuta la preparación y genera splits + `tasks.json`.
  - `verify_processed_splits` — comprueba que los CSVs `train/val/test` existen por *run*.
- Define rutas base:
  - `RAW = data/raw/udacity`
  - `PROC = data/processed`

> Esta celda **no ejecuta** el *prep* aún; sólo deja listo el entorno.

[↑ Volver al índice](#toc)

In [5]:
# %% setup & imports
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys, json
import pandas as pd

ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.prep.data_prep import PrepConfig, run_prep, verify_processed_splits
RAW  = ROOT / "data" / "raw" / "udacity"
PROC = ROOT / "data" / "processed"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a id="sec-02"></a>

## 2) Parámetros de la preparación

**Objetivo:** declarar los *runs* y la configuración del *split*.

- `RUNS`: lista de recorridos a procesar (ej. `["circuito1", "circuito2"]`).
- `use_left_right=False`: modo QC/split básico, **sin** expansión de cámaras.  
  > Si quieres triplicar datos con *left/right* y corrección de `steer_shift`, usa `01A_PREP_BALANCED.ipynb` o cambia aquí a `True` (no recomendado en este notebook de QC).
- `bins`: nº de contenedores para estratificar por `steering`.
- `train/val`: proporciones del split (test se infiere).
- `seed`: reproducibilidad del split.

> `target_per_bin` y `cap_per_bin` **no se usan** en este notebook (se usan en el balanceo offline de `01A_PREP_BALANCED`).

[↑ Volver al índice](#toc)

In [6]:
# Parámetros de la preparación
RUNS = ["circuito1", "circuito2"]

CFG = PrepConfig(
    root=ROOT,
    runs=RUNS,
    use_left_right=False,  # QC + split básico (sin expansión)
    steer_shift=0.2,       # no aplica si use_left_right=False
    bins=21,
    train=0.70,
    val=0.15,
    seed=42,
    target_per_bin=None,   # <- no balancea aquí
    cap_per_bin=None,
)
CFG

PrepConfig(root=PosixPath('/home/cesar/proyectos/TFM_SNN'), runs=['circuito1', 'circuito2'], use_left_right=False, steer_shift=0.2, bins=21, train=0.7, val=0.15, seed=42, target_per_bin=None, cap_per_bin=None)

<a id="sec-03"></a>

## 3) Ejecutar *prep* y verificar

**Objetivo:** correr el pipeline y validar que los *splits* existen.

Acciones que realiza `run_prep(CFG)`:
- Lee y normaliza `driving_log.csv` → `canonical.csv`.
- Genera *splits* estratificados: `train.csv`, `val.csv`, `test.csv`.
- Construye/actualiza `data/processed/tasks.json` con la lista de *runs* y rutas a sus CSVs.

Luego, `verify_processed_splits(PROC, RUNS)`:
- Comprueba que `train/val/test.csv` existen para cada *run*.
- Si falta algo, lanza una excepción con detalle.

> **Idempotencia:** si vuelves a ejecutar esta celda, se regenerarán los CSVs y se actualizará `tasks.json` de forma consistente con lo que haya ahora en `RUNS`.

[↑ Volver al índice](#toc)

In [7]:
# Ejecutar prep + verificar
manifest = run_prep(CFG)
print("OK:", PROC/"prep_manifest.json")

# Verificación: existen train/val/test por run
verify_processed_splits(PROC, RUNS)
print("OK: splits 'train/val/test' encontrados.")


OK: /home/cesar/proyectos/TFM_SNN/data/processed/prep_manifest.json
OK: splits 'train/val/test' encontrados.


<a id="sec-04"></a>

## 4) Resumen de rutas de *splits* (vista rápida)

**Objetivo:** visualizar de un vistazo qué ficheros usará el pipeline *continual*.

- Lee `data/processed/tasks.json`.
- Muestra una tabla con `run`, `train_csv`, `val_csv` y `test_csv`.

> Si los paths no son los esperados, revisa la config de la celda 2 o el estado de `data/raw/udacity/<run>/`.

[↑ Volver al índice](#toc)

In [8]:
# Resumen de rutas de splits (visualización)
tasks_json = json.loads((PROC/"tasks.json").read_text(encoding="utf-8"))
pd.DataFrame({
    "run": tasks_json["tasks_order"],
    "train_csv": [tasks_json["splits"][r]["train"] for r in tasks_json["tasks_order"]],
    "val_csv":   [tasks_json["splits"][r]["val"]   for r in tasks_json["tasks_order"]],
    "test_csv":  [tasks_json["splits"][r]["test"]  for r in tasks_json["tasks_order"]],
})


,run,train_csv,val_csv,test_csv
0,circuito1,/home/cesar/proyectos/TFM_SNN/data/processed/c...,/home/cesar/proyectos/TFM_SNN/data/processed/c...,/home/cesar/proyectos/TFM_SNN/data/processed/c...
1,circuito2,/home/cesar/proyectos/TFM_SNN/data/processed/c...,/home/cesar/proyectos/TFM_SNN/data/processed/c...,/home/cesar/proyectos/TFM_SNN/data/processed/c...
